In [1]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [95]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import catboost as cat
from catboost import CatBoostClassifier

In [96]:
# CSV de treino

df_train = pd.read_csv("flight_delays_train.csv")
df_train

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


In [97]:
# CSV de Teste

df_teste = pd.read_csv("flight_delays_test.csv")
df_teste

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258
...,...,...,...,...,...,...,...,...
99995,c-6,c-5,c-2,852,WN,CRP,HOU,187
99996,c-11,c-24,c-6,1446,UA,ORD,LAS,1515
99997,c-1,c-30,c-2,1509,OO,ORD,SGF,438
99998,c-1,c-5,c-5,804,DL,LGA,ATL,761


In [98]:
df_train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [99]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  object
 1   DayofMonth         100000 non-null  object
 2   DayOfWeek          100000 non-null  object
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  object
 5   Origin             100000 non-null  object
 6   Dest               100000 non-null  object
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


## Precisei mapear para que eu pudesse entender e usar as colunas de Dest, Origin e UniqueCarrier

In [100]:
# Mapeando os pontos de origem e destino

ori_dest = pd.Series((df_teste['Origin'].append(df_teste['Dest']).append(df_train['Origin']).append(
                    df_train['Dest'])).unique()).to_dict()
ori_dest_map = {v: k for k, v in ori_dest.items()}
ori_dest_map

{'MRY': 0,
 'LAS': 1,
 'GSP': 2,
 'BWI': 3,
 'ORD': 4,
 'ORF': 5,
 'IAD': 6,
 'MSP': 7,
 'PIT': 8,
 'DFW': 9,
 'JFK': 10,
 'ASE': 11,
 'GRR': 12,
 'DEN': 13,
 'DAB': 14,
 'ATL': 15,
 'CMH': 16,
 'MDT': 17,
 'IAH': 18,
 'MIA': 19,
 'LAX': 20,
 'TYS': 21,
 'MCI': 22,
 'STL': 23,
 'LGA': 24,
 'DTW': 25,
 'CLT': 26,
 'DCA': 27,
 'SFO': 28,
 'MSN': 29,
 'IND': 30,
 'ALB': 31,
 'HNL': 32,
 'PWM': 33,
 'MDW': 34,
 'PHX': 35,
 'BOS': 36,
 'GEG': 37,
 'ABE': 38,
 'DAL': 39,
 'EWR': 40,
 'LEX': 41,
 'OAK': 42,
 'SAN': 43,
 'SGF': 44,
 'OMA': 45,
 'MAF': 46,
 'CLD': 47,
 'SNA': 48,
 'BUR': 49,
 'PHL': 50,
 'PDX': 51,
 'BDL': 52,
 'ANC': 53,
 'JAN': 54,
 'OKC': 55,
 'MLU': 56,
 'RDU': 57,
 'ONT': 58,
 'SLC': 59,
 'TWF': 60,
 'FLL': 61,
 'ABI': 62,
 'CLE': 63,
 'CVG': 64,
 'AZO': 65,
 'SWF': 66,
 'SJU': 67,
 'COS': 68,
 'GSO': 69,
 'MEM': 70,
 'MCO': 71,
 'RSW': 72,
 'TPA': 73,
 'FAT': 74,
 'BUF': 75,
 'JAX': 76,
 'ITO': 77,
 'ERI': 78,
 'PIA': 79,
 'LIT': 80,
 'SBA': 81,
 'CID': 82,
 'BHM': 83,
 '

In [101]:
# Mapeando as transportadoras
# Correios = UniqueCarriers

correios = pd.Series((df_teste['UniqueCarrier'].append(df_teste['UniqueCarrier']).append(
                    df_train['UniqueCarrier']).append(df_train['UniqueCarrier'])).unique()).to_dict()
correios_map = {v: k for k, v in correios.items()}
correios

{0: 'YV',
 1: 'WN',
 2: 'MQ',
 3: 'UA',
 4: 'NW',
 5: 'B6',
 6: 'US',
 7: 'AA',
 8: 'OH',
 9: 'OO',
 10: 'FL',
 11: 'DL',
 12: 'EV',
 13: 'CO',
 14: 'XE',
 15: '9E',
 16: 'HA',
 17: 'AS',
 18: 'AQ',
 19: 'F9',
 20: 'DH',
 21: 'TZ',
 22: 'HP'}

In [102]:
# Tirar as colunas Object

def new_df(df):
    df_copy = df.copy()
    df_copy['Dest'] = df_copy['Dest'].map(ori_dest_map)
    df_copy['Origin'] = df_copy['Origin'].map(ori_dest_map)
    df_copy['UniqueCarrier'] = df_copy['UniqueCarrier'].map(correios_map)
    df_copy['Month'] = df_copy['Month'].str.replace('c-', '').astype(int)
    df_copy['DayofMonth'] = df_copy['DayofMonth'].str.replace('c-', '').astype(int)
    df_copy['DayOfWeek'] = df_copy['DayOfWeek'].str.replace('c-', '').astype(int)
    return df_copy


# ############################################


# train['DayOfWeek'] = train.DayOfWeek.apply(lambda x: int(x.split('-')[1]))
# train['Month'] = train.Month.apply(lambda x: int(x.split('-')[1]))
# train['DayofMonth'] = train.DayofMonth.apply(lambda x: int(x.split('-')[1]))

# train['dep_delayed_15min'] = train.dep_delayed_15min.apply(lambda x: 1 if x == 'Y' else 0)

# origin_list = train.Origin.value_counts().head(10).index.tolist()
# train['Origin'] = train.Origin.apply(lambda x: x if x in origin_list else 'other')

# dest_list = train.Dest.value_counts().head(10).index.tolist()
# train['Dest'] = train.Dest.apply(lambda x: x if x in dest_list else 'other')

# carriers_list = train.UniqueCarrier.value_counts().head(10).index.tolist()
# train['UniqueCarrier'] = train.UniqueCarrier.apply(lambda x: x if x in carriers_list else 'other')

# train = pd.get_dummies(train)

### Confirmando mudança na tabela de treino

In [119]:
# Confirmando a mudança das tabelas para "int" na tabela de treino

new_df(df_train)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,8,21,7,1934,7,15,9,732,N
1,4,20,3,1548,6,8,71,834,N
2,9,2,5,1422,14,57,63,416,N
3,11,25,6,1015,9,13,70,872,N
4,10,7,6,1828,1,34,45,423,Y
...,...,...,...,...,...,...,...,...,...
99995,5,4,3,1618,9,28,250,199,N
99996,1,18,3,804,13,40,14,884,N
99997,1,24,2,1901,4,25,18,1076,N
99998,4,27,4,1515,2,9,239,140,N


In [104]:
# Confirmando a mudança das tabelas para "int"

new_df(df_train).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Month              100000 non-null  int64 
 1   DayofMonth         100000 non-null  int64 
 2   DayOfWeek          100000 non-null  int64 
 3   DepTime            100000 non-null  int64 
 4   UniqueCarrier      100000 non-null  int64 
 5   Origin             100000 non-null  int64 
 6   Dest               100000 non-null  int64 
 7   Distance           100000 non-null  int64 
 8   dep_delayed_15min  100000 non-null  object
dtypes: int64(8), object(1)
memory usage: 6.9+ MB


### Confirmando mudança na tabela de teste

In [105]:
# Confirmando a mudança das tabelas para "int" na tabela de teste

new_df(df_teste)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,7,25,3,615,0,0,35,598
1,4,17,2,739,1,1,127,1235
2,12,2,7,651,2,2,4,577
3,3,25,7,1614,1,3,121,377
4,6,6,3,1505,3,4,23,258
...,...,...,...,...,...,...,...,...
99995,6,5,2,852,1,181,127,187
99996,11,24,6,1446,3,4,1,1515
99997,1,30,2,1509,9,4,44,438
99998,1,5,5,804,11,24,15,761


In [106]:
# Confirmando a mudança das tabelas para "int" na tabela de teste

new_df(df_teste).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   Month          100000 non-null  int64
 1   DayofMonth     100000 non-null  int64
 2   DayOfWeek      100000 non-null  int64
 3   DepTime        100000 non-null  int64
 4   UniqueCarrier  100000 non-null  int64
 5   Origin         100000 non-null  int64
 6   Dest           100000 non-null  int64
 7   Distance       100000 non-null  int64
dtypes: int64(8)
memory usage: 6.1 MB


In [123]:
# Declarando target
x_train, y_train = new_df(df_train), new_df(df_train)['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values

x = new_df(df_train).drop("dep_delayed_15min", axis=1)
y = new_df(df_train)["dep_delayed_15min"]

In [124]:
# Splitando os dados e mapeando a coluna target

x_train, x_teste, y_train, y_teste = train_test_split(x, y, test_size = 0.3, random_state = 42)


In [125]:
# Catboost parameters tuning
from sklearn.model_selection import GridSearchCV

lista_parametros = {'depth': [4, 7, 10],
                  'learning_rate' : [0.03, 0.1, 0.15],
                 'l2_leaf_reg': [1,4,9],
                'iterations': [300]}

new_cat = cat.CatBoostClassifier()
cat_model = GridSearchCV(new_cat, lista_parametros, scoring = "roc_auc", cv = 3, n_jobs= -1)
cat_model.fit(x_train, y_train)

print("Melhores parâmetros:")
print()
print(cat_model.best_params_)
print()
print(cat_model.best_score_)

0:	learn: 0.6408291	total: 88.1ms	remaining: 26.3s
1:	learn: 0.6011406	total: 106ms	remaining: 15.7s
2:	learn: 0.5693623	total: 124ms	remaining: 12.3s
3:	learn: 0.5440421	total: 140ms	remaining: 10.3s
4:	learn: 0.5257781	total: 156ms	remaining: 9.2s
5:	learn: 0.5110294	total: 173ms	remaining: 8.5s
6:	learn: 0.4994329	total: 193ms	remaining: 8.09s
7:	learn: 0.4896110	total: 212ms	remaining: 7.75s
8:	learn: 0.4806579	total: 238ms	remaining: 7.7s
9:	learn: 0.4732882	total: 255ms	remaining: 7.41s
10:	learn: 0.4680326	total: 271ms	remaining: 7.12s
11:	learn: 0.4639526	total: 287ms	remaining: 6.89s
12:	learn: 0.4609596	total: 305ms	remaining: 6.74s
13:	learn: 0.4572804	total: 321ms	remaining: 6.55s
14:	learn: 0.4550429	total: 337ms	remaining: 6.4s
15:	learn: 0.4533068	total: 352ms	remaining: 6.25s
16:	learn: 0.4515435	total: 368ms	remaining: 6.12s
17:	learn: 0.4498733	total: 384ms	remaining: 6.02s
18:	learn: 0.4484615	total: 400ms	remaining: 5.91s
19:	learn: 0.4473557	total: 416ms	remaining:

171:	learn: 0.4124677	total: 3.01s	remaining: 2.24s
172:	learn: 0.4123643	total: 3.03s	remaining: 2.22s
173:	learn: 0.4122514	total: 3.05s	remaining: 2.21s
174:	learn: 0.4121256	total: 3.06s	remaining: 2.19s
175:	learn: 0.4120023	total: 3.08s	remaining: 2.17s
176:	learn: 0.4118832	total: 3.1s	remaining: 2.15s
177:	learn: 0.4117544	total: 3.11s	remaining: 2.13s
178:	learn: 0.4116102	total: 3.13s	remaining: 2.12s
179:	learn: 0.4114484	total: 3.15s	remaining: 2.1s
180:	learn: 0.4113217	total: 3.17s	remaining: 2.08s
181:	learn: 0.4112218	total: 3.19s	remaining: 2.07s
182:	learn: 0.4110435	total: 3.21s	remaining: 2.05s
183:	learn: 0.4109533	total: 3.23s	remaining: 2.03s
184:	learn: 0.4108311	total: 3.24s	remaining: 2.02s
185:	learn: 0.4107185	total: 3.26s	remaining: 2s
186:	learn: 0.4106180	total: 3.28s	remaining: 1.98s
187:	learn: 0.4104009	total: 3.29s	remaining: 1.96s
188:	learn: 0.4102325	total: 3.31s	remaining: 1.95s
189:	learn: 0.4100837	total: 3.33s	remaining: 1.93s
190:	learn: 0.409

In [132]:
# Rodando com os melhores parâmetros

new_cat = cat.CatBoostClassifier(depth=7, iterations=300, l2_leaf_reg=9, learning_rate=0.1)
new_cat.fit(x_train, y_train)

# Fazendo o predict
prediction = new_cat.predict_proba(x_teste)[:, 1]

print()
print(roc_auc_score(y_teste, prediction))

0:	learn: 0.6408291	total: 30.5ms	remaining: 9.12s
1:	learn: 0.6011406	total: 50.1ms	remaining: 7.46s
2:	learn: 0.5693623	total: 65.9ms	remaining: 6.52s
3:	learn: 0.5440421	total: 81.3ms	remaining: 6.02s
4:	learn: 0.5257781	total: 97.6ms	remaining: 5.76s
5:	learn: 0.5110294	total: 118ms	remaining: 5.8s
6:	learn: 0.4994329	total: 138ms	remaining: 5.77s
7:	learn: 0.4896110	total: 156ms	remaining: 5.71s
8:	learn: 0.4806579	total: 172ms	remaining: 5.56s
9:	learn: 0.4732882	total: 188ms	remaining: 5.44s
10:	learn: 0.4680326	total: 204ms	remaining: 5.35s
11:	learn: 0.4639526	total: 221ms	remaining: 5.3s
12:	learn: 0.4609596	total: 252ms	remaining: 5.56s
13:	learn: 0.4572804	total: 272ms	remaining: 5.55s
14:	learn: 0.4550429	total: 289ms	remaining: 5.48s
15:	learn: 0.4533068	total: 305ms	remaining: 5.42s
16:	learn: 0.4515435	total: 324ms	remaining: 5.39s
17:	learn: 0.4498733	total: 341ms	remaining: 5.34s
18:	learn: 0.4484615	total: 358ms	remaining: 5.3s
19:	learn: 0.4473557	total: 375ms	remai

161:	learn: 0.4138034	total: 3.7s	remaining: 3.15s
162:	learn: 0.4136827	total: 3.72s	remaining: 3.13s
163:	learn: 0.4135216	total: 3.75s	remaining: 3.11s
164:	learn: 0.4134404	total: 3.77s	remaining: 3.09s
165:	learn: 0.4133010	total: 3.79s	remaining: 3.06s
166:	learn: 0.4131947	total: 3.82s	remaining: 3.04s
167:	learn: 0.4130246	total: 3.85s	remaining: 3.02s
168:	learn: 0.4128514	total: 3.88s	remaining: 3s
169:	learn: 0.4126740	total: 3.92s	remaining: 2.99s
170:	learn: 0.4125800	total: 3.95s	remaining: 2.98s
171:	learn: 0.4124677	total: 3.98s	remaining: 2.96s
172:	learn: 0.4123643	total: 4.01s	remaining: 2.94s
173:	learn: 0.4122514	total: 4.04s	remaining: 2.92s
174:	learn: 0.4121256	total: 4.06s	remaining: 2.9s
175:	learn: 0.4120023	total: 4.09s	remaining: 2.88s
176:	learn: 0.4118832	total: 4.13s	remaining: 2.87s
177:	learn: 0.4117544	total: 4.16s	remaining: 2.85s
178:	learn: 0.4116102	total: 4.19s	remaining: 2.83s
179:	learn: 0.4114484	total: 4.22s	remaining: 2.81s
180:	learn: 0.411

### Base de teste

In [135]:
# Rodando agora na base de teste

new_cat.fit(x_train, y_train)
prediction_teste = new_cat.predict_proba(new_df(df_teste))[:, 1]

0:	learn: 0.6408291	total: 18ms	remaining: 5.37s
1:	learn: 0.6011406	total: 35.9ms	remaining: 5.35s
2:	learn: 0.5693623	total: 52.5ms	remaining: 5.19s
3:	learn: 0.5440421	total: 68.1ms	remaining: 5.04s
4:	learn: 0.5257781	total: 84.2ms	remaining: 4.97s
5:	learn: 0.5110294	total: 102ms	remaining: 4.99s
6:	learn: 0.4994329	total: 118ms	remaining: 4.93s
7:	learn: 0.4896110	total: 134ms	remaining: 4.9s
8:	learn: 0.4806579	total: 151ms	remaining: 4.87s
9:	learn: 0.4732882	total: 167ms	remaining: 4.85s
10:	learn: 0.4680326	total: 186ms	remaining: 4.89s
11:	learn: 0.4639526	total: 206ms	remaining: 4.94s
12:	learn: 0.4609596	total: 226ms	remaining: 4.98s
13:	learn: 0.4572804	total: 250ms	remaining: 5.11s
14:	learn: 0.4550429	total: 266ms	remaining: 5.06s
15:	learn: 0.4533068	total: 282ms	remaining: 5.01s
16:	learn: 0.4515435	total: 299ms	remaining: 4.97s
17:	learn: 0.4498733	total: 315ms	remaining: 4.94s
18:	learn: 0.4484615	total: 332ms	remaining: 4.91s
19:	learn: 0.4473557	total: 348ms	remai

162:	learn: 0.4136827	total: 3.57s	remaining: 3s
163:	learn: 0.4135216	total: 3.61s	remaining: 2.99s
164:	learn: 0.4134404	total: 3.63s	remaining: 2.97s
165:	learn: 0.4133010	total: 3.66s	remaining: 2.96s
166:	learn: 0.4131947	total: 3.69s	remaining: 2.94s
167:	learn: 0.4130246	total: 3.71s	remaining: 2.91s
168:	learn: 0.4128514	total: 3.74s	remaining: 2.9s
169:	learn: 0.4126740	total: 3.76s	remaining: 2.88s
170:	learn: 0.4125800	total: 3.78s	remaining: 2.85s
171:	learn: 0.4124677	total: 3.8s	remaining: 2.83s
172:	learn: 0.4123643	total: 3.82s	remaining: 2.81s
173:	learn: 0.4122514	total: 3.84s	remaining: 2.78s
174:	learn: 0.4121256	total: 3.86s	remaining: 2.76s
175:	learn: 0.4120023	total: 3.88s	remaining: 2.74s
176:	learn: 0.4118832	total: 3.91s	remaining: 2.72s
177:	learn: 0.4117544	total: 3.93s	remaining: 2.7s
178:	learn: 0.4116102	total: 3.96s	remaining: 2.67s
179:	learn: 0.4114484	total: 3.98s	remaining: 2.65s
180:	learn: 0.4113217	total: 4s	remaining: 2.63s
181:	learn: 0.4112218

In [136]:
pd.Series(prediction_teste, name = "dep_delayed_15min").to_csv("resposta_final.csv", index_label = "id", 
                                                               header = True)